In [ ]:
%matplotlib inline

# General packages for system, time, etc
import os, time, csv, sys
import datetime
from datetime import date
import glob

# scitnific computing and plotting
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

# HDDM related packages
import pymc as pm
import hddm
import kabuki
import arviz as az
print("The current HDDM version is: ", hddm.__version__)
print("The current kabuki version is: ", kabuki.__version__)
print("The current PyMC version is: ", pm.__version__)
print("The current ArviZ version is: ", az.__version__)

# parallel processing related
from p_tqdm import p_map
from functools import partial

from sklearn.metrics import r2_score

# 1.Run Model

In [ ]:
path=os.getcwd()
data=pd.read_csv(path+'/allData.csv')

In [ ]:
path=os.getcwd()+'/del_r2negative/'
f_list = os.listdir(path)
idx=0
m_idx,m_file=[],[]
for i in f_list:
    if os.path.splitext(i)[1]  != '.db':
            m_idx.append(idx)
            m_file.append(i)
    idx+=1

In [ ]:
left_sub=[]
for i in m_file:
    left_sub.append(int(i.split('_')[2]))
left_sub.sort()

In [ ]:
new_data=data.copy()
for i in range(1,134):
    if i not in left_sub:
        new_data.drop(new_data[new_data['subj_idx'] == i].index, inplace=True)
new_data['rt']=new_data['rt']/1000

In [ ]:
new_data.to_csv('left_sub.csv')

In [ ]:
def m1_id(id, df=None, samples=None, burn=None, thin=1, save_name="m1"): 
    """
    This function defines a function that run a HDDM model
    id — the id of a cpu thread
    data — The input data
    samples — number of samples for MCMC
    burn — number of burn in (or warm-up) of MCMC
    thin — number of thin, the same as in HDDM
    save_name — prefix of file name when saving the model objects.
    """
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id    
    m      = hddm.HDDM(df, depends_on = {'v':'expConds','a':'expConds','t':'expConds'},include={'v','a','t','sv','st'},
                     p_outlier = .05,is_group_model=True,bias=False)
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle')
    m.save(mname)
    
    return m

In [ ]:
nsample=10000
burns=2000
thins=2
chains = 4

In [ ]:
%%time

file_names = glob.glob("m1_id_tmp" + "_chain_*[!db]", recursive=False)

if file_names:
    file_names = sorted(file_names, key=lambda x: x[-1]) # sort filenames by chain ID
    m1res = []
    for fname in file_names:
        print('current loading: ', fname, '\n')
        m1res.append(hddm.load(fname))
else:
    m1res = p_map(partial(m1_id, df=new_data, samples=nsample, burn=burns, thin=thins,save_name="m1_id_tmp"), range(chains))

# 2.Load Model

In [ ]:
file_names = glob.glob("m1_id_tmp" + "_chain_*[!db]", recursive=False)
m1=[]
for f in file_names:
    m1.append(hddm.load(f))

# 3.Model convergence

#### R-hat 指标检查模型是否拟合好了，所有参数的R-hat<1.01 则表明模型拟合好了

In [ ]:
from kabuki.analyze import gelman_rubin
gelman_rubin(m1)

In [ ]:
np.max(list(gelman_rubin(m1).values()))

#### combine these three models to get a better approximation of the posterior distribution.

In [ ]:
# Combine the models we ran to test for convergence.
m = kabuki.utils.concat_models(m1)

### visual trace

In [ ]:
m.plot_posteriors(save=True)

# 4.Compute R^2

In [ ]:
m.values

In [ ]:
scsv=pd.DataFrame([m.values])
scsv.to_csv('del_negative_group_model_param.csv')

In [ ]:
#modelt_1.csv
filename = 'del_negative_group_model_param.csv'
model_excel = pd.read_csv(filename)
#simulate data for group ourside
ename=['easy_absent_cued','easy_absent_uncued','easy_present_cued','easy_present_uncued',
      'hard_absent_cued','hard_absent_uncued','hard_present_cued','hard_present_uncued']
pname=['v','v_std','a','a_std','t','t_std','sv','st']
params={}
for e in ename:
    params[e]={}
    for p in range(len(pname)):
        if p==0 or p==2 or p==4:
            this_name=pname[p]+'('+e+')'
        else:
            this_name=pname[p]  
        params[e][pname[p]]=model_excel[this_name][0]
        params[e]['z']=0.5

## false & real data

In [ ]:
simulated = hddm.generate.gen_rand_data(params,size=10000)
simulated_data=simulated[0]

In [ ]:
simulated_data

In [ ]:
fdata,rdata={},{}
for e in ename:
    fdata[e]=simulated_data[simulated_data['condition']==e]
    rdata[e]=new_data[new_data['expConds']==e]

In [ ]:
fdata_rt,rdata_rt={},{}
fdata_acc,rdata_acc={},{}
for e in ename:
    fdata_rt[e] = np.mean(fdata[e]['rt'])
    rdata_rt[e] = np.mean(rdata[e]['rt'])
    fdata_acc[e]=float(np.sum(fdata[e]['response']==1))/len(fdata[e])
    rdata_acc[e]=float(np.sum(rdata[e]['response']==1))/len(rdata[e])

In [ ]:
## plot data distribution

import numpy as np

fd_r,rd_r,fd_a,rd_a=[],[],[],[]
for e in ename:
    fd_r.append(fdata_rt[e])
    rd_r.append(rdata_rt[e])
    fd_a.append(fdata_acc[e])
    rd_a.append(rdata_acc[e])

## rt
fig,ax = plt.subplots(figsize=(20,10),dpi=80)
width_1 = 0.4
ax.bar(np.arange(len(fd_r)),fd_r,width=width_1,tick_label=ename,label = "false data")
ax.bar(np.arange(len(rd_r))+width_1,rd_r,width=width_1,tick_label=ename,label="real data")
ax.legend()
plt.ylabel('RT (ms)')
plt.show()

In [ ]:
## acc
fig,ax = plt.subplots(figsize=(20,10),dpi=80)
width_1 = 0.4
ax.bar(np.arange(len(fd_a)),fd_a,width=width_1,tick_label=ename,label = "false data")
ax.bar(np.arange(len(rd_a))+width_1,rd_a,width=width_1,tick_label=ename,label="real data")
ax.legend()
plt.ylabel('ACC (%)')
plt.show()

In [ ]:
print('r2_score rt:   ',  r2_score(np.array(rd_r), np.array(fd_r)))
print('r2_score acc:  ',  r2_score(np.array(rd_a), np.array(fd_a)))

# 5.Plot and Compare 参数在不同条件下的后验分布，得到不同参数在不同条件下的差异，以及是否显著

## Plot mzw posterior

In [ ]:
from utils import interpolate_trace
from matplotlib.pylab import figure
def mzw_plot_posterior_nodes(traces, names,bins=50, lb=None, ub=None):
    """Plot interpolated posterior of a list of nodes.

    :Arguments:
        nodes : list of pymc.Node's
            List of pymc.Node's to plot the posterior of.
            These can be found in model.nodes_db.node.loc['param_name']
        bins : int (default=50)
            How many bins to use for computing the histogram.
        lb : float (default is to infer from data)
            Lower boundary to use for plotting.
        ub : float (default is to infer from data)
            Upper boundary to use for plotting.
    """
    figure(figsize=(4,4))
    if lb is None:
        lb = np.min(traces)
    if ub is None:
        ub = np.max(traces)

    x_data = np.linspace(lb, ub, 300)

    for i in range(len(names)):
        trace=traces[:,i]
        # hist = interpolate_trace(x_data, trace, range=(trace.min(), trace.max()), bins=bins)
        hist = interpolate_trace(x_data, trace, range=(lb, ub), bins=bins)
        plt.plot(x_data, hist, label=names[i], lw=2.0)

    leg = plt.legend(loc="upper right", fancybox=True)
    leg.get_frame().set_alpha(0.5)

### (1) Drift rate

In [ ]:
v_ebc, v_ebu, v_epc, v_epu, v_hbc, v_hbu, v_hpc, v_hpu = m.nodes_db.node[['v(easy_absent_cued)','v(easy_absent_uncued)','v(easy_present_cued)','v(easy_present_uncued)',
                                                                         'v(hard_absent_cued)','v(hard_absent_uncued)','v(hard_present_cued)','v(hard_present_uncued)']]

hddm.analyze.plot_posterior_nodes([v_ebc, v_ebu, v_epc, v_epu, v_hbc, v_hbu, v_hpc, v_hpu])
plt.xlabel('Drift rate')
plt.ylabel('Posterior probability')
plt.title('Posterior of group means')
plt.savefig('Drift rate Posterior probability.png'，dpi=300)

#### Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([v_ebc.trace(),v_epc.trace(),v_hbc.trace(),v_hpc.trace()],axis=0),1)
uncued_trace=np.expand_dims(np.mean([v_ebu.trace(),v_epu.trace(),v_hbu.trace(),v_hpu.trace()],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
mzw_plot_posterior_nodes(traces,['Cued','Uncued'])
plt.xlabel('Drift rate')
plt.ylabel('Posterior probability')
plt.title('Cueing effect')
plt.savefig('Drift rate Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((v_ebc.trace()+v_epc.trace()+v_hbc.trace()+v_hpc.trace()) > (v_ebu.trace()+v_epu.trace()+v_hbu.trace()+v_hpu.trace())).mean())

#### Task*Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([(v_ebc.trace()-v_ebu.trace()),(v_epc.trace()-v_epu.trace())],axis=0),1)
uncued_trace=np.expand_dims(np.mean([(v_hbc.trace()-v_hbu.trace()),(v_hpc.trace()-v_hpu.trace())],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
mzw_plot_posterior_nodes(traces,['Easy','Hard'])
plt.xlabel('Drift rate')
plt.ylabel('Cued - Uncued\nPosterior probability')
plt.title('Task difficulty*Cueing')
plt.savefig('Drift rate Task-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((v_ebc.trace()-v_ebu.trace()+v_epc.trace()-v_epu.trace()) < (v_hbc.trace()-v_hbu.trace()+v_hpc.trace()-v_hpu.trace())).mean())

#### Central*Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([(v_ebc.trace()-v_ebu.trace()),(v_hbc.trace()-v_hbu.trace())],axis=0),1)
uncued_trace=np.expand_dims(np.mean([(v_epc.trace()-v_epu.trace()),(v_hpc.trace()-v_hpu.trace())],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
traces=traces*1000
mzw_plot_posterior_nodes(traces,['Absent','Present'])
plt.xlabel('Drift rate')
plt.ylabel('Cued - Uncued\nPosterior probabilit')
plt.title('Central Fixation*Cueing')
plt.savefig('Drift rate Central-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((v_ebc.trace()-v_ebu.trace()+v_hbc.trace()-v_hbu.trace()) < (v_epc.trace()-v_epu.trace()+v_hpc.trace()-v_hpu.trace())).mean())

#### Task * Central * Cueing

In [ ]:
ea=np.expand_dims(np.mean([(v_ebc.trace()-v_ebu.trace())],axis=0),1)
ep=np.expand_dims(np.mean([(v_epc.trace()-v_epu.trace())],axis=0),1)
ha=np.expand_dims(np.mean([(v_hbc.trace()-v_hbu.trace())],axis=0),1)
hp=np.expand_dims(np.mean([(v_hpc.trace()-v_hpu.trace())],axis=0),1)

traces=np.concatenate([ea,ep,ha,hp],axis=1)
mzw_plot_posterior_nodes(traces,['Easy-Absent','Easy-Present','Hard-Absent','Hard-present'])
plt.xlabel('Drift rate')
plt.ylabel('Cued - Uncued\nPosterior probability')
plt.title('Task difficulty*Central Fixation*Cueing')
plt.savefig('Drift rate Task-Central-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
#print('Significance:',((a_ebc.trace()-a_ebu.trace()+a_hbc.trace()-a_hbu.trace()) < (a_epc.trace()-a_epu.trace()+a_hpc.trace()-a_hpu.trace())).mean())

### (2) Non-decision time

In [ ]:
t_ebc, t_ebu, t_epc, t_epu, t_hbc, t_hbu, t_hpc, t_hpu = m.nodes_db.node[['t(easy_absent_cued)','t(easy_absent_uncued)','t(easy_present_cued)','t(easy_present_uncued)',
                                                                         't(hard_absent_cued)','t(hard_absent_uncued)','t(hard_present_cued)','t(hard_present_uncued)']]

hddm.analyze.plot_posterior_nodes([t_ebc, t_ebu, t_epc, t_epu, t_hbc, t_hbu, t_hpc, t_hpu])
plt.xlabel('Non-decision time (s)')
plt.ylabel('Posterior probability')
plt.title('Posterior of group means')
plt.savefig('Non-decision time Posterior probability.png')

#### Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([t_ebc.trace(),t_epc.trace(),t_hbc.trace(),t_hpc.trace()],axis=0),1)
uncued_trace=np.expand_dims(np.mean([t_ebu.trace(),t_epu.trace(),t_hbu.trace(),t_hpu.trace()],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
mzw_plot_posterior_nodes(traces,['Cued','Uncued'])
traces=traces*1000
plt.xlabel('Non-decision time (ms)')
plt.ylabel('Posterior probability')
plt.title('Cueing effect')
plt.savefig('Non-decision time Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((t_ebc.trace()+t_epc.trace()+t_hbc.trace()+t_hpc.trace()) < (t_ebu.trace()+t_epu.trace()+t_hbu.trace()+t_hpu.trace())).mean())

#### Task*Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([(t_ebc.trace()-t_ebu.trace()),(t_epc.trace()-t_epu.trace())],axis=0),1)
uncued_trace=np.expand_dims(np.mean([(t_hbc.trace()-t_hbu.trace()),(t_hpc.trace()-t_hpu.trace())],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
traces=traces*1000
mzw_plot_posterior_nodes(traces,['Easy','Hard'])
plt.xlabel('Non-decision time (ms)')
plt.ylabel('Cued - Uncued\nPosterior probability')
plt.title('Task difficulty*Cueing')
plt.savefig('Non-decision time Task-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((t_ebc.trace()-t_ebu.trace()+t_epc.trace()-t_epu.trace()) > (t_hbc.trace()-t_hbu.trace()+t_hpc.trace()-t_hpu.trace())).mean())

#### Central*Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([(t_ebc.trace()-t_ebu.trace()),(t_hbc.trace()-t_hbu.trace())],axis=0),1)
uncued_trace=np.expand_dims(np.mean([(t_epc.trace()-t_epu.trace()),(t_hpc.trace()-t_hpu.trace())],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
traces=traces*1000
mzw_plot_posterior_nodes(traces,['Absent','Present'])
plt.xlabel('Non-decision time (ms)')
plt.ylabel('Cued - Uncued\nPosterior probabilit')
plt.title('Central Fixation*Cueing')
plt.savefig('Non-decision time Central-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((t_ebc.trace()-t_ebu.trace()+t_hbc.trace()-t_hbu.trace()) > (t_epc.trace()-t_epu.trace()+t_hpc.trace()-t_hpu.trace())).mean())

#### Task * Central * Cueing

In [ ]:
ea=np.expand_dims(np.mean([(t_ebc.trace()-t_ebu.trace())],axis=0),1)
ep=np.expand_dims(np.mean([(t_epc.trace()-t_epu.trace())],axis=0),1)
ha=np.expand_dims(np.mean([(t_hbc.trace()-t_hbu.trace())],axis=0),1)
hp=np.expand_dims(np.mean([(t_hpc.trace()-t_hpu.trace())],axis=0),1)

traces=np.concatenate([ea,ep,ha,hp],axis=1)
traces=traces*1000
mzw_plot_posterior_nodes(traces,['Easy-Absent','Easy-Present','Hard-Absent','Hard-present'])
plt.xlabel('Non-decision time (ms)')
plt.ylabel('Cued - Uncued\nPosterior probability')
plt.title('Task difficulty*Central Fixation*Cueing')
plt.savefig('Non-decision time Task-Central-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
#print('Significance:',((a_ebc.trace()-a_ebu.trace()+a_hbc.trace()-a_hbu.trace()) < (a_epc.trace()-a_epu.trace()+a_hpc.trace()-a_hpu.trace())).mean())

### (3) Boundary

In [ ]:
a_ebc, a_ebu, a_epc, a_epu, a_hbc, a_hbu, a_hpc, a_hpu = m.nodes_db.node[['a(easy_absent_cued)','a(easy_absent_uncued)','a(easy_present_cued)','a(easy_present_uncued)',
                                                                         'a(hard_absent_cued)','a(hard_absent_uncued)','a(hard_present_cued)','a(hard_present_uncued)']]

hddm.analyze.plot_posterior_nodes([a_ebc, a_ebu, a_epc, a_epu, a_hbc, a_hbu, a_hpc, a_hpu])
plt.xlabel('Boundary')
plt.ylabel('Posterior probability')
plt.title('Posterior of group means')
plt.savefig('Boundary Posterior probability.png')

#### Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([a_ebc.trace(),a_epc.trace(),a_hbc.trace(),a_hpc.trace()],axis=0),1)
uncued_trace=np.expand_dims(np.mean([a_ebu.trace(),a_epu.trace(),a_hbu.trace(),a_hpu.trace()],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
mzw_plot_posterior_nodes(traces,['Cued','Uncued'])
plt.xlabel('Boundary')
plt.ylabel('Posterior probability')
plt.title('Cueing effect')
plt.savefig('Boundary Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((a_ebc.trace()+a_epc.trace()+a_hbc.trace()+a_hpc.trace()) > (a_ebu.trace()+a_epu.trace()+a_hbu.trace()+a_hpu.trace())).mean())

#### Task*Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([(a_ebc.trace()-a_ebu.trace()),(a_epc.trace()-a_epu.trace())],axis=0),1)
uncued_trace=np.expand_dims(np.mean([(a_hbc.trace()-a_hbu.trace()),(a_hpc.trace()-a_hpu.trace())],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
mzw_plot_posterior_nodes(traces,['Easy','Hard'])
plt.xlabel('Boundary')
plt.ylabel('Cued - Uncued\nPosterior probability')
plt.title('Task difficulty*Cueing')
plt.savefig('Boundary Task-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((a_ebc.trace()-a_ebu.trace()+a_epc.trace()-a_epu.trace()) < (a_hbc.trace()-a_hbu.trace()+a_hpc.trace()-a_hpu.trace())).mean())

#### Central*Cueing

In [ ]:
cued_trace=np.expand_dims(np.mean([(a_ebc.trace()-a_ebu.trace()),(a_hbc.trace()-a_hbu.trace())],axis=0),1)
uncued_trace=np.expand_dims(np.mean([(a_epc.trace()-a_epu.trace()),(a_hpc.trace()-a_hpu.trace())],axis=0),1)

traces=np.concatenate([cued_trace,uncued_trace],axis=1)
mzw_plot_posterior_nodes(traces,['Absent','Present'])
plt.xlabel('Boundary')
plt.ylabel('Cued - Uncued\nPosterior probability')
plt.title('Central Fixation*Cueing')
plt.savefig('Boundary Central-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
print('Significance:',((a_ebc.trace()-a_ebu.trace()+a_hbc.trace()-a_hbu.trace()) < (a_epc.trace()-a_epu.trace()+a_hpc.trace()-a_hpu.trace())).mean())

#### Task * Central * Cueing

In [ ]:
ea=np.expand_dims(np.mean([(a_ebc.trace()-a_ebu.trace())],axis=0),1)
ep=np.expand_dims(np.mean([(a_epc.trace()-a_epu.trace())],axis=0),1)
ha=np.expand_dims(np.mean([(a_hbc.trace()-a_hbu.trace())],axis=0),1)
hp=np.expand_dims(np.mean([(a_hpc.trace()-a_hpu.trace())],axis=0),1)

traces=np.concatenate([ea,ep,ha,hp],axis=1)
mzw_plot_posterior_nodes(traces,['Easy-Absent','Easy-Present','Hard-Absent','Hard-present'])
plt.xlabel('Boundary')
plt.ylabel('Cued - Uncued\nPosterior probability')
plt.title('Task difficulty*Central Fixation*Cueing')
plt.savefig('Boundary Task-Central-Cueing Posterior probability.png',dpi=300,bbox_inches='tight')
#print('Significance:',((a_ebc.trace()-a_ebu.trace()+a_hbc.trace()-a_hbu.trace()) < (a_epc.trace()-a_epu.trace()+a_hpc.trace()-a_hpu.trace())).mean())